In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Collection

In [3]:
import pandas as pd
from sodapy import Socrata

# Create the Socrata client
client = Socrata("data.cityofnewyork.us", None)  # Replace `None` with your app token if available

# Dataset URL
pay_url = 'k397-673e'

def extract_all_data(pay_url, chunks=100000):
    """
    Extract all data from a Socrata dataset in chunks and return a consolidated DataFrame.

    Parameters:
        pay_url (str): The dataset identifier on Socrata.
        chunks (int): The number of records to retrieve per chunk.

    Returns:
        pd.DataFrame: The concatenated DataFrame containing all extracted records.
    """
    offset = 0
    pay_url_chunks = []

    while True:
        # Fetch a chunk of data
        results = client.get(pay_url, limit=chunks, offset=offset)

        # Break if no more data
        if not results:
            break

        # Append the chunk as a DataFrame
        pay_url_chunks.append(pd.DataFrame(results))

        # Update the offset
        offset += chunks
        print(f"Fetched {offset} records...")  # Progress indicator

    # Concatenate all chunks into a single DataFrame
    pay_df = pd.concat(pay_url_chunks, ignore_index=True) if pay_url_chunks else pd.DataFrame()
    return pay_df

# Extract all data
payroll_data = extract_all_data(pay_url=pay_url)



Fetched 100000 records...
Fetched 200000 records...
Fetched 300000 records...
Fetched 400000 records...
Fetched 500000 records...
Fetched 600000 records...
Fetched 700000 records...
Fetched 800000 records...
Fetched 900000 records...
Fetched 1000000 records...
Fetched 1100000 records...
Fetched 1200000 records...
Fetched 1300000 records...
Fetched 1400000 records...
Fetched 1500000 records...
Fetched 1600000 records...
Fetched 1700000 records...
Fetched 1800000 records...
Fetched 1900000 records...
Fetched 2000000 records...
Fetched 2100000 records...
Fetched 2200000 records...
Fetched 2300000 records...
Fetched 2400000 records...
Fetched 2500000 records...
Fetched 2600000 records...
Fetched 2700000 records...
Fetched 2800000 records...
Fetched 2900000 records...
Fetched 3000000 records...
Fetched 3100000 records...
Fetched 3200000 records...
Fetched 3300000 records...
Fetched 3400000 records...
Fetched 3500000 records...
Fetched 3600000 records...
Fetched 3700000 records...
Fetched 38

## Data Cleaning

In [4]:
payroll_data.shape

(6225611, 17)

In [ ]:
payroll_data.head()

,fiscal_year,payroll_number,agency_name,last_name,first_name,mid_init,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay
0,2024,67,ADMIN FOR CHILDREN'S SVCS,THOMAS,MANJU,S,2012-04-16T00:00:00.000,MANHATTAN,AGENCY ATTORNEY,ACTIVE,129176.00,per Annum,1820.00,135435.84,6.75,583.87,15073.66
1,2024,67,ADMIN FOR CHILDREN'S SVCS,HOWARD,CONNIE,T,2011-07-04T00:00:00.000,BRONX,YOUTH DEVELOPMENT SPECIALIST,ACTIVE,67667.00,per Annum,1820.00,67653.94,786,42954.65,15084.17
2,2024,67,ADMIN FOR CHILDREN'S SVCS,OGUNRIBIDO,RUFUS,O,2011-07-04T00:00:00.000,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,ACTIVE,67596.00,per Annum,1820.00,67583.22,1171.75,66982.56,15177.12
3,2024,67,ADMIN FOR CHILDREN'S SVCS,EROMOSELE,PETER,O,2011-07-04T00:00:00.000,BROOKLYN,ASSOCIATE YOUTH DEVELOPMENT SPECIALIST,ACTIVE,90561.00,per Annum,1815.25,86721.82,956.25,68471.27,15183.75
4,2024,67,ADMIN FOR CHILDREN'S SVCS,WILLIAMS,PAUL,B,2023-04-24T00:00:00.000,BRONX,SPECIAL OFFICER,ACTIVE,36955.00,per Annum,2080.00,36403.26,553.87,14985.45,15246.50


In [5]:
payroll_data[['base_salary', 'regular_hours', 'regular_gross_paid', 'ot_hours', 'total_ot_paid', 'total_other_pay']] = payroll_data[['base_salary', 'regular_hours', 'regular_gross_paid', 'ot_hours', 'total_ot_paid', 'total_other_pay']].astype(float)


In [6]:
payroll_data['agency_start_date'] = pd.to_datetime(payroll_data['agency_start_date'],errors='coerce')

In [7]:
payroll_data = payroll_data.dropna(subset=["agency_start_date"])


In [8]:
payroll_data['start_year'] = payroll_data['agency_start_date'].dt.year

In [9]:
df = payroll_data.copy()
df["pay_basis"] = df["pay_basis"].str.strip()


## Exploratory Data Analysis

In [12]:
df['start_year'] = df['start_year'].astype(int)
df['fiscal_year'] = df['fiscal_year'].astype(int)

df['years_worked'] = df['fiscal_year'] - df['start_year']

In [13]:
df = df[(df['base_salary'] > 10000) & (df['regular_gross_paid'] > 10000)]


In [14]:
#create new col less than 1-5 years, 6-10, 11-20, 21-30, 31-40, 40+



def label_age(row):
    if row['years_worked'] < 5:
        return 'less than 5 years'
    elif 5 <= row['years_worked'] <= 10:
        return '6 - 10 years'
    elif 11 <= row['years_worked'] <= 20:
        return '11 - 20 years'
    elif 21 <= row['years_worked'] <= 30:
        return '21 - 30 years'
    elif 31 <= row['years_worked'] <= 40:
        return '31 - 40 years'
    else:
        return '40+ years'

df['years_worked_cat'] = df.apply(label_age, axis=1)


/var/folders/_r/82gxfcq11h968vttjmtdt7lh0000gn/T/ipykernel_39039/1613633017.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_worked_cat'] = df.apply(label_age, axis=1)


In [15]:
df['hourly_rate'] = df['base_salary'] / df['regular_hours']

/var/folders/_r/82gxfcq11h968vttjmtdt7lh0000gn/T/ipykernel_39039/2963067656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hourly_rate'] = df['base_salary'] / df['regular_hours']


In [16]:
df.head()

,fiscal_year,payroll_number,agency_name,last_name,first_name,mid_init,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,start_year,years_worked,years_worked_cat,hourly_rate
0,2024,67,ADMIN FOR CHILDREN'S SVCS,THOMAS,MANJU,S,2012-04-16,MANHATTAN,AGENCY ATTORNEY,ACTIVE,129176.0,per Annum,1820.00,135435.84,6.75,583.87,15073.66,2012,12,11 - 20 years,70.975824
1,2024,67,ADMIN FOR CHILDREN'S SVCS,HOWARD,CONNIE,T,2011-07-04,BRONX,YOUTH DEVELOPMENT SPECIALIST,ACTIVE,67667.0,per Annum,1820.00,67653.94,786.00,42954.65,15084.17,2011,13,11 - 20 years,37.179670
2,2024,67,ADMIN FOR CHILDREN'S SVCS,OGUNRIBIDO,RUFUS,O,2011-07-04,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,ACTIVE,67596.0,per Annum,1820.00,67583.22,1171.75,66982.56,15177.12,2011,13,11 - 20 years,37.140659
3,2024,67,ADMIN FOR CHILDREN'S SVCS,EROMOSELE,PETER,O,2011-07-04,BROOKLYN,ASSOCIATE YOUTH DEVELOPMENT SPECIALIST,ACTIVE,90561.0,per Annum,1815.25,86721.82,956.25,68471.27,15183.75,2011,13,11 - 20 years,49.888996
4,2024,67,ADMIN FOR CHILDREN'S SVCS,WILLIAMS,PAUL,B,2023-04-24,BRONX,SPECIAL OFFICER,ACTIVE,36955.0,per Annum,2080.00,36403.26,553.87,14985.45,15246.50,2023,1,less than 5 years,17.766827


###  DuckDB for faster querying

In [17]:
import duckdb
con = duckdb.connect()

In [ ]:
sql = """
    SUMMARIZE
        SELECT *
        FROM df
"""

# summary statistics relation
filtered_df_new = con.sql(sql)

# push to pandas df for readability
filtered_df_new.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,fiscal_year,INTEGER,2014,2024,12,2019.0134573684327,3.1234536798617754,2016,2019,2022,3410325,0.00
1,payroll_number,VARCHAR,10,996,159,None,None,None,None,None,3410325,28.84
2,agency_name,VARCHAR,ADMIN FOR CHILDREN'S SVCS,TECHNOLOGY & INNOVATION,145,None,None,None,None,None,3410325,0.00
3,last_name,VARCHAR,1,xxxx,163267,None,None,None,None,None,3410325,0.25
4,first_name,VARCHAR,A,xxx,54835,None,None,None,None,None,3410325,0.25
5,mid_init,VARCHAR,(,x,36,None,None,None,None,None,3410325,39.27
6,agency_start_date,TIMESTAMP_NS,1901-01-01 00:00:00,2024-08-19 00:00:00,15753,None,None,2000-04-04 09:37:41.351949,2007-09-23 21:27:27.327007,2014-12-12 19:25:07.159308,3410325,0.00
7,work_location_borough,VARCHAR,ALBANY,WESTCHESTER,22,None,None,None,None,None,3410325,8.45
8,title_description,VARCHAR,*ADM DIR FLEET MAINT-MGRL ASGMNT,YOUTH DEVELOPMENT SPECIALIST,1538,None,None,None,None,None,3410325,0.00
9,leave_status_as_of_june_30,VARCHAR,ACTIVE,SEASONAL,5,None,None,None,None,None,3410325,0.00


I noticed inconsistent data

In [19]:

df = df.drop(df[df.years_worked > 69].index)

### Max Years Worked by Agency

In [ ]:
sql = """
SELECT agency_name, MAX(years_worked) as max_years FROM df GROUP BY agency_name Order By max_years DESC LIMIT 15"""


temp = con.sql(sql).df()

temp

,agency_name,max_years
0,OFFICE OF THE COMPTROLLER,69
1,DEPT OF ED PEDAGOGICAL,67
2,HRA/DEPT OF SOCIAL SERVICES,67
3,DEPARTMENT OF CITY PLANNING,65
4,DEPARTMENT OF EDUCATION ADMIN,63
5,DEPARTMENT OF SANITATION,62
6,HOUSING PRESERVATION & DVLPMNT,61
7,DEPT OF HEALTH/MENTAL HYGIENE,60
8,POLICE DEPARTMENT,60
9,DEPARTMENT OF CORRECTION,58


### Average Salary by Agency

In [ ]:
duckdb.sql("SELECT agency_name, ROUND(AVG(base_salary),2) as Average_Salary_by_Dept FROM df GROUP BY agency_name Order BY Average_Salary_by_Dept DESC LIMIT 10;")


┌────────────────────────────────┬────────────────────────┐
│          agency_name           │ Average_Salary_by_Dept │
│            varchar             │         double         │
├────────────────────────────────┼────────────────────────┤
│ OFFICE OF RACIAL EQUITY        │              151092.64 │
│ COMMISSION ON RACIAL EQUITY    │               136191.0 │
│ DISTRICTING COMMISSION         │              120641.34 │
│ OFFICE OF CRIMINAL JUSTICE     │               115668.1 │
│ OFFICE OF COLLECTIVE BARGAININ │              114714.84 │
│ FINANCIAL INFO SVCS AGENCY     │              110957.06 │
│ OFFICE OF THE ACTUARY          │              106908.74 │
│ BRONX COMMUNITY BOARD #3       │              105813.81 │
│ PUBLIC ADMINISTRATOR-RICHMOND  │              105751.75 │
│ MUNICIPAL WATER FIN AUTHORITY  │              101299.87 │
├────────────────────────────────┴────────────────────────┤
│ 10 rows                                       2 columns │
└───────────────────────────────────────

### Avg Salary by Employees per Fiscal Year

In [ ]:
duckdb.sql("SELECT fiscal_year, ROUND(AVG(base_salary),2) as Average_Salary_by_FY FROM df GROUP BY fiscal_year Order BY fiscal_year DESC LIMIT 10;")


┌─────────────┬──────────────────────┐
│ fiscal_year │ Average_Salary_by_FY │
│   varchar   │        double        │
├─────────────┼──────────────────────┤
│ 2024        │             53706.85 │
│ 2023        │              51329.9 │
│ 2022        │              48405.8 │
│ 2021        │             46966.23 │
│ 2020        │             46546.49 │
│ 2019        │             44636.08 │
│ 2018        │             45506.67 │
│ 2017        │             42856.08 │
│ 2016        │             43153.79 │
│ 2015        │             39158.45 │
├─────────────┴──────────────────────┤
│ 10 rows                  2 columns │
└────────────────────────────────────┘

### Avg Salary per Fiscal Year Grouped by Agency

In [ ]:
duckdb.sql("SELECT fiscal_year ,agency_name, ROUND(AVG(base_salary),2) as Average_Salary FROM df GROUP BY fiscal_year, agency_name Order BY fiscal_year DESC LIMIT 10;")


┌─────────────┬────────────────────────────────┬────────────────┐
│ fiscal_year │          agency_name           │ Average_Salary │
│   varchar   │            varchar             │     double     │
├─────────────┼────────────────────────────────┼────────────────┤
│ 2024        │ EQUAL EMPLOY PRACTICES COMM    │        89298.4 │
│ 2024        │ BROOKLYN COMMUNITY BOARD #6    │       71698.83 │
│ 2024        │ BOARD OF ELECTION POLL WORKERS │           1.01 │
│ 2024        │ QUEENS COMMUNITY BOARD #2      │       67907.93 │
│ 2024        │ QUEENS COMMUNITY BOARD #14     │      110242.67 │
│ 2024        │ QUEENS COMMUNITY BOARD #13     │       112445.5 │
│ 2024        │ QUEENS COMMUNITY BOARD #12     │       74916.67 │
│ 2024        │ QUEENS COMMUNITY BOARD #8      │        52184.0 │
│ 2024        │ STATEN ISLAND COMMUNITY BD #1  │       100885.5 │
│ 2024        │ PUBLIC ADVOCATE                │       62215.14 │
├─────────────┴────────────────────────────────┴────────────────┤
│ 10 rows 

### Avg Salary by Work Location

In [ ]:
duckdb.sql("SELECT work_location_borough,  ROUND(AVG(base_salary),2) as Average_Salary FROM df GROUP BY work_location_borough Order By Average_Salary DESC")

┌───────────────────────┬────────────────┐
│ work_location_borough │ Average_Salary │
│        varchar        │     double     │
├───────────────────────┼────────────────┤
│ OTHER                 │      113633.54 │
│ WASHINGTON DC         │      113261.45 │
│ ULSTER                │       75242.68 │
│ ALBANY                │       73080.56 │
│ Queens                │       71758.65 │
│ Richmond              │       69804.47 │
│ SULLIVAN              │       68174.57 │
│ SCHOHARIE             │       67683.22 │
│ WESTCHESTER           │       67138.62 │
│ PUTNAM                │       64164.38 │
│ Manhattan             │       62936.85 │
│ GREENE                │       62752.93 │
│ DELAWARE              │       62144.06 │
│ Bronx                 │       58788.14 │
│ RICHMOND              │       56055.92 │
│ BROOKLYN              │       55601.21 │
│ DUTCHESS              │       55506.46 │
│ QUEENS                │       54810.81 │
│ BRONX                 │       51757.71 │
│ ORANGE   

In [20]:
import plotly.express as px
sql = """
SELECT work_location_borough,  ROUND(AVG(base_salary),2) as Average_Salary FROM df GROUP BY work_location_borough Order By Average_Salary DESC"""

temp = con.sql(sql).df()


fig = px.bar(
    temp,
    x="work_location_borough",
    y="Average_Salary",
    color_discrete_sequence=["#1f77b4"],  # Change bar color
)

# Customize x-axis and y-axis titles
fig.update_layout(
    xaxis_title="Work Location",  # X-axis label
    yaxis_title="Average Salary (USD)",  # Y-axis label
    title="Average Salary by Work Location"  # Chart title
)

# Show the chart
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### Average Salary by Title/Job

In [ ]:
duckdb.sql("SELECT title_description,  ROUND(AVG(base_salary),2) as Average_Salary FROM df GROUP BY title_description Order By Average_Salary DESC LIMIT 15")

┌───────────────────────────────────────────────────────────┬────────────────┐
│                     title_description                     │ Average_Salary │
│                          varchar                          │     double     │
├───────────────────────────────────────────────────────────┼────────────────┤
│ CHIEF ACTUARY                                             │      296470.38 │
│ PENSION INVESTMENT ADVISOR                                │       295813.8 │
│ CHANCELLOR                                                │       285292.5 │
│ CAPTAIN - CHIEF OF STAFF                                  │       276588.0 │
│ FIRST DEPUTY MAYOR                                        │      274919.07 │
│ CHIEF OF STAFF TO THE MAYOR                               │       269822.5 │
│ President                                                 │       263394.0 │
│ DIRECTOR OF THE MAYOR?S OFFICE OF ADMINISTRATIVE SERVICES │       260043.0 │
│ DIRECTOR OF THE MAYOR?S OFFICE OF OPERATIONS      

### Avg Overtime Hours Per Agency

In [ ]:
duckdb.sql("SELECT agency_name,  ROUND(AVG(ot_hours)) as Avg_OT FROM df GROUP BY agency_name Order By Avg_OT DESC LIMIT 15")

┌────────────────────────────────┬────────┐
│          agency_name           │ Avg_OT │
│            varchar             │ double │
├────────────────────────────────┼────────┤
│ FIRE DEPARTMENT                │  353.0 │
│ DEPARTMENT OF CORRECTION       │  315.0 │
│ BOARD OF ELECTION              │  259.0 │
│ POLICE DEPARTMENT              │  216.0 │
│ DEPARTMENT OF SANITATION       │  215.0 │
│ Police Department              │  193.0 │
│ DEPT OF CITYWIDE ADMIN SVCS    │  189.0 │
│ DEPARTMENT OF TRANSPORTATION   │  184.0 │
│ NYC HOUSING AUTHORITY          │  173.0 │
│ DEPT. OF HOMELESS SERVICES     │  162.0 │
│ ADMIN FOR CHILDREN'S SVCS      │  120.0 │
│ DEPT OF ENVIRONMENT PROTECTION │  118.0 │
│ HRA/DEPT OF SOCIAL SERVICES    │   86.0 │
│ DEPARTMENT OF BUILDINGS        │   82.0 │
│ NYC EMPLOYEES RETIREMENT SYS   │   81.0 │
├────────────────────────────────┴────────┤
│ 15 rows                       2 columns │
└─────────────────────────────────────────┘

### Avg Total Overtime Paid By Agency

In [ ]:
duckdb.sql("SELECT agency_name,  ROUND(AVG(total_ot_paid)) as Avg_OT_paid FROM df GROUP BY agency_name Order By Avg_OT_paid DESC LIMIT 15")

┌────────────────────────────────┬─────────────┐
│          agency_name           │ Avg_OT_paid │
│            varchar             │   double    │
├────────────────────────────────┼─────────────┤
│ FIRE DEPARTMENT                │     20874.0 │
│ DEPARTMENT OF CORRECTION       │     17866.0 │
│ DEPARTMENT OF SANITATION       │     13083.0 │
│ POLICE DEPARTMENT              │     12737.0 │
│ DEPARTMENT OF TRANSPORTATION   │     11513.0 │
│ Police Department              │     10354.0 │
│ DEPT OF CITYWIDE ADMIN SVCS    │      9455.0 │
│ BOARD OF ELECTION              │      9223.0 │
│ NYC HOUSING AUTHORITY          │      9000.0 │
│ DEPT OF ENVIRONMENT PROTECTION │      7031.0 │
│ DEPT. OF HOMELESS SERVICES     │      6706.0 │
│ ADMIN FOR CHILDREN'S SVCS      │      5392.0 │
│ DEPARTMENT OF BUILDINGS        │      4101.0 │
│ OFFICE OF EMERGENCY MANAGEMENT │      3911.0 │
│ HRA/DEPT OF SOCIAL SERVICES    │      3554.0 │
├────────────────────────────────┴─────────────┤
│ 15 rows           

### Avg Overtime Hours by title/job

In [ ]:
duckdb.sql("SELECT title_description,  ROUND(AVG(ot_hours)) as Avg_OT FROM df GROUP BY title_description Order By Avg_OT DESC LIMIT 15")

┌──────────────────────────────────────────────┬────────┐
│              title_description               │ Avg_OT │
│                   varchar                    │ double │
├──────────────────────────────────────────────┼────────┤
│ SENIOR SYSTEMS ANALYSTS                      │  874.0 │
│ EXECUTIVE DEPUTY SHERIFF                     │  823.0 │
│ SENIOR INSTITUTIONAL TRADES INSTRUCTOR       │  796.0 │
│ SUPERVISOR DOCKBUILDER                       │  779.0 │
│ CHIEF MARINE ENGINEER                        │  776.0 │
│ MARINE ENGINEER                              │  754.0 │
│ PILOT                                        │  683.0 │
│ MARINE OILER                                 │  677.0 │
│ PLANNER: PRODUCTION CNTRL & SCHEDULING       │  667.0 │
│ SUPERVISOR OF BRIDGE OPERATIONS              │  667.0 │
│ SUPERVISOR SHIP CARPENTER                    │  637.0 │
│ SUPERVISOR SHEET METAL WORKER                │  603.0 │
│ WARDEN-ASSISTANT DEPUTY WARDEN TED < 11/1/92 │  597.0 │
│ ASSOCIATE JU

### Avg Years Worked by Title


In [ ]:
duckdb.sql("SELECT title_description,  ROUND(AVG(years_worked)) as Avg_Years_Worked FROM df GROUP BY title_description Order By Avg_Years_Worked DESC LIMIT 15")

┌─────────────────────────────────────────────┬──────────────────┐
│              title_description              │ Avg_Years_Worked │
│                   varchar                   │      double      │
├─────────────────────────────────────────────┼──────────────────┤
│ ?WARDEN                                     │             53.0 │
│ SPVSG FAD-ASST COMMISS DETAIL               │             51.0 │
│ F/T FILM INSPECTION ASST.                   │             50.0 │
│ RESEARCH SCIENTIST                          │             46.0 │
│ Assistant Commissioner                      │             46.0 │
│ ?*ELEVATOR OPERATOR                         │             45.0 │
│ ASSISTANT ACCOUNTANT                        │             44.0 │
│ NURSE'S AIDE                                │             44.0 │
│ HIGHER EDUCATION OFFICER/ASST ADMINISTRATOR │             44.0 │
│ CALENDAR ASSISTANT                          │             43.0 │
│ MENTAL HEALTH WORKER                        │             42

In [ ]:
sql = """
SELECT title_description,  ROUND(AVG(years_worked)) as Avg_Years_Worked FROM df GROUP BY title_description Order By Avg_Years_Worked DESC LIMIT 15"""


temp = con.sql(sql).df()


fig = px.bar(temp, x='title_description', y='Avg_Years_Worked', text_auto='.2s')

fig.update_layout(
    xaxis_title="Title Description",  # X-axis label
    yaxis_title="Average Years Worked",  # Y-axis label
    title="Average Years Worked by Title"  # Chart title
)
fig.show()



### Avg Year Worker Per Agency

In [ ]:
duckdb.sql("SELECT agency_name,  ROUND(AVG(years_worked)) as Avg_Years_Worked FROM df GROUP BY agency_name Order By Avg_Years_Worked DESC LIMIT 20")


┌────────────────────────────────┬──────────────────┐
│          agency_name           │ Avg_Years_Worked │
│            varchar             │      double      │
├────────────────────────────────┼──────────────────┤
│ BRONX COMMUNITY BOARD #3       │             24.0 │
│ BROOKLYN COMMUNITY BOARD #7    │             23.0 │
│ BROOKLYN COMMUNITY BOARD #1    │             23.0 │
│ QUEENS COMMUNITY BOARD #5      │             23.0 │
│ STATEN ISLAND COMMUNITY BD #1  │             21.0 │
│ BROOKLYN COMMUNITY BOARD #16   │             21.0 │
│ QUEENS COMMUNITY BOARD #4      │             20.0 │
│ BROOKLYN COMMUNITY BOARD #8    │             18.0 │
│ QUEENS COMMUNITY BOARD #14     │             18.0 │
│ PUBLIC ADMINISTRATOR-NEW YORK  │             18.0 │
│ BRONX COMMUNITY BOARD #1       │             17.0 │
│ BRONX COMMUNITY BOARD #4       │             16.0 │
│ QUEENS COMMUNITY BOARD #12     │             16.0 │
│ BROOKLYN COMMUNITY BOARD #4    │             16.0 │
│ DOE CUSTODIAL PAYROL      

### Max Salary Per Agency

In [ ]:
sql = """
SELECT agency_name, MAX(base_salary) as max_salary FROM df GROUP BY agency_name Order By max_salary DESC LIMIT 15"""


temp = con.sql(sql).df()



In [ ]:
import plotly.express as px

fig = px.bar(temp, x='agency_name', y='max_salary', text_auto='.2s')

fig.update_layout(
    xaxis_title="Agency",  # X-axis label
    yaxis_title="Max Salary",  # Y-axis label
    title="Max Salary by Department"
)

fig.show()



### Scatter Plot Showing the Base Salary & OT Paid Per Agency

In [ ]:
import plotly.express as px

# Filter the DataFrame to include only the desired agencies
filtered_df = df[df['agency_name'].isin(['OFFICE OF RACIAL EQUITY', 'COMMISSION ON RACIAL EQUITY', 'OFFICE OF COLLECTIVE BARGAININ',
                                         'DISTRICTING COMMISSION', 'OFFICE OF CRIMINAL JUSTICE', 'DOE CUSTODIAL PAYROL',
                                         'FINANCIAL INFO SVCS AGENCY', 'OFFICE OF THE ACTUARY', 'TECHNOLOGY & INNOVATION', 'OFFICE OF THE MAYOR'])]

# Create the scatter plot
fig = px.scatter(
    filtered_df,
    x="total_ot_paid",
    y="base_salary",
    color="years_worked_cat",
    facet_col="agency_name",  # Use filtered values for facet columns
)

# Update layout to rotate facet labels
fig.update_layout(
    annotations=[
        dict(
            text=anno['text'],
            x=anno['x'],
            y=anno['y'],
            xanchor=anno['xanchor'],
            yanchor=anno['yanchor'],
            showarrow=False,
            textangle=15,
            font=dict(size=10)
        )
        for anno in fig['layout']['annotations']
    ]
)

# Show the plot
fig.show()



### Hiring Rates

In [ ]:
sql = """
SELECT CAST(start_year AS STRING) AS start_year, COUNT(last_name) AS count_last_name
FROM df
GROUP BY CAST(start_year AS STRING)
ORDER BY start_year DESC
LIMIT 15
"""

temp = con.sql(sql).df()

temp



,start_year,count_last_name
0,2024,4891
1,2023,29005
2,2022,41129
3,2021,53850
4,2020,40777
5,2019,100212
6,2018,112246
7,2017,125124
8,2016,173427
9,2015,167297


In [ ]:
hiring_rates = df.groupby('start_year').size().reset_index(name='num_hires')

In [ ]:
hiring_rates['pct_change'] = hiring_rates['num_hires'].pct_change() * 100

In [ ]:
hiring_rates_sorted = hiring_rates.sort_values(by='start_year', ascending=False).head(25)

In [ ]:
import plotly.express as px
fig = px.line(hiring_rates_sorted, x="start_year", y="pct_change", title='Year-over-Year Percentage Change in Hires', markers=True)
fig.show()


### Hourly Rates Based on Years Worked

In [ ]:
import numpy as np
numeric_cols = df.select_dtypes(include=[np.number])

df[numeric_cols.columns] = df[numeric_cols.columns].replace([np.inf, -np.inf], np.nan)

df = df[~np.isinf(numeric_cols).any(axis=1)]


In [ ]:
result = df.groupby('years_worked_cat')['hourly_rate'].mean().reset_index()
result = result.sort_values(by='hourly_rate')

In [ ]:
import plotly.express as px

fig = px.bar(result, x='years_worked_cat', y='hourly_rate',text_auto='.2s', title='Hourly Rate Based on Years Worked')

fig.update_layout(
    xaxis_title="Years Worked Category",
    yaxis_title="Hourly Rate (USD)",

)
fig.update_traces(
    textposition="outside"
)


fig.show()
